# EDINETから「主な相手先別の販売実績」を抽出する

import libraries

In [1]:
!pip install edinet_xbrl

In [2]:
import requests
import pandas as pd
import zipfile
import os
from glob import glob
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
from typing import Dict, List
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup

EDINET API より財務情報の一覧を取得

In [3]:
END_POINT = 'https://disclosure.edinet-fsa.go.jp/api/v1'
submission_info_endpoint = f'{END_POINT}/documents.json'
submission_request_parameters = {
    'date': '2022-03-31',
    'type': 2
}
submission_info_response = requests.get(
    submission_info_endpoint, submission_request_parameters
)
submission_info_json = submission_info_response.json()

In [4]:
raw_submission_info_df = pd.DataFrame(submission_info_json['results'])
raw_submission_info_df.columns

Index(['seqNumber', 'docID', 'edinetCode', 'secCode', 'JCN', 'filerName',
       'fundCode', 'ordinanceCode', 'formCode', 'docTypeCode', 'periodStart',
       'periodEnd', 'submitDateTime', 'docDescription', 'issuerEdinetCode',
       'subjectEdinetCode', 'subsidiaryEdinetCode', 'currentReportReason',
       'parentDocID', 'opeDateTime', 'withdrawalStatus', 'docInfoEditStatus',
       'disclosureStatus', 'xbrlFlag', 'pdfFlag', 'attachDocFlag',
       'englishDocFlag', 'csvFlag', 'legalStatus'],
      dtype='object')

In [5]:
# 重要なカラムに絞る
submission_info_df = raw_submission_info_df[['docID', 'edinetCode', 'secCode', 'filerName', 'docDescription']]
submission_info_df.head()

,docID,edinetCode,secCode,filerName,docDescription
0,S100NSP1,E00196,18990,株式会社福田組,臨時報告書
1,S100NJAS,E12460,None,野村アセットマネジメント株式会社,有価証券届出書（内国投資信託受益証券）
2,S100NQG1,None,None,None,None
3,S100NJAR,E12460,None,野村アセットマネジメント株式会社,有価証券報告書（内国投資信託受益証券）－第17期(令和3年7月7日－令和4年1月6日)
4,S100NSTT,E36150,None,株式会社エスティエンジニアリング,変更報告書


In [6]:
# 有価証券報告書の情報を抽出する。
securities_report_infos = []
for i, row in submission_info_df.iterrows():
    doc_desc = row['docDescription']
    
    if doc_desc is None:
        continue
    
    if ('有価証券報告書' in doc_desc) and ('受益証券' not in doc_desc) and ('訂正' not in doc_desc):
        row_to_dataframe = pd.DataFrame([row])
        securities_report_infos.append(row_to_dataframe)

if len(securities_report_infos) == 0:
    print('有価証券報告書の提出情報がありません。')
else:
    print(f'{len(securities_report_infos)} 件の有価証券報告書が抽出されました。')
    securities_report_info_df = pd.concat(securities_report_infos)

102 件の有価証券報告書が抽出されました。


In [7]:
securities_report_info_df

,docID,edinetCode,secCode,filerName,docDescription
5,S100NRFI,E02331,77160,株式会社ナカニシ,有価証券報告書－第70期(令和3年1月1日－令和3年12月31日)
8,S100NR8K,E02898,76090,ダイトロン株式会社,有価証券報告書－第70期(令和3年1月1日－令和3年12月31日)
72,S100NSAP,E04163,None,長崎自動車株式会社,有価証券報告書－第125期(令和3年1月1日－令和3年12月31日)
74,S100NST4,E02445,59650,株式会社フジマック,有価証券報告書－第73期(令和3年1月1日－令和3年12月31日)
81,S100NSKK,E01659,59460,株式会社長府製作所,有価証券報告書－第68期(令和3年1月1日－令和3年12月31日)
...,...,...,...,...,...
732,S100NT0D,E05453,37580,株式会社アエリア,有価証券報告書－第20期(令和3年1月1日－令和3年12月31日)
735,S100NTB3,E05241,27210,株式会社ジェイホールディングス,有価証券報告書－第30期(令和3年1月1日－令和3年12月31日)
744,S100NSKH,E37115,44150,株式会社ブロードエンタープライズ,有価証券報告書－第22期(令和3年1月1日－令和3年12月31日)
746,S100NSHB,E00393,25010,サッポロホールディングス株式会社,有価証券報告書－第98期(令和3年1月1日－令和3年12月31日)


In [8]:
securities_report_info_df[securities_report_info_df['docID'] == 'S100NT30']

,docID,edinetCode,secCode,filerName,docDescription
555,S100NT30,E05951,93990,ビート・ホールディングス・リミテッド （貝德控股有限公司、Ｂｅａｔ Ｈｏｌｄｉｎｇｓ Ｌｉｍ...,有価証券報告書－第18期(令和3年1月1日－令和3年12月31日)


In [9]:
# 今回は〇〇の第〇期決算情報を対象とする。
docID = 'S100NT30'
label = securities_report_info_df[securities_report_info_df['docID'] == docID]
document_endpoint = f'{END_POINT}/documents/{docID}'
document_request_parameters = {
    'type': 1
}
document_response = requests.get(document_endpoint, document_request_parameters)

In [10]:
# まず、返ってきたデータを zip 形式で保存する。
zip_file_full_path = f'D:/EDINET_DATA/{docID}.zip'
with open(zip_file_full_path, 'wb') as f:
    for chunk in document_response.iter_content(chunk_size=1024):
        f.write(chunk)

In [11]:
# zip ファイルを解凍する
os.makedirs(f'D:/EDINET_DATA/{docID}', exist_ok=True)
with zipfile.ZipFile(zip_file_full_path) as zip_f:
    zip_f.extractall(f'D:/EDINET_DATA/{docID}')

In [12]:
# xbrl ファイルを発見する
# PublicDoc 内に格納されている xbrl ファイルが分析対象となるファイルである。
xbrl_expression = f'D:/EDINET_DATA/{docID}/**/PublicDoc/**/*.xbrl'
xbrl_paths = glob(xbrl_expression, recursive=True)
xbrl_paths

['D:/EDINET_DATA/S100NT30\\XBRL\\PublicDoc\\jpcrp080000-asr-001_E05951-000_2021-12-31_01_2022-03-31.xbrl']

In [13]:
parser = EdinetXbrlParser()
# Step2で特定した XBRL ファイルのパスを選択
xbrl_path = xbrl_paths[0]
parsed_xbrl = parser.parse_file(xbrl_path)

c:\Users\hisan\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [14]:
# 有価証券報告書には基本5年分の情報があるため、5年分の情報をとってくる。

durations = [
    'CurrentYearDuration',
    'Prior1YearDuration',
    'Prior2YearDuration',
    'Prior3YearDuration',
    'Prior4YearDuration'
]

def translate_period(report_end_date: str, duration: str) -> str:
    '''duration を対応する日付情報に変換'''
    report_end_date_datetime = datetime.strptime(report_end_date, '%Y-%m-%d')
    if duration == 'CurrentYearDuration':
        return report_end_date_datetime.strftime("%Y-%m-%d")
    else:
        n_year = int(duration[5])
        n_year_previous_date = report_end_date_datetime - relativedelta(years=n_year)
        return n_year_previous_date.strftime("%Y-%m-%d")

def get_all_periods_value(
    xbrl: EdinetXbrlParser,
    report_end_date: str,
    key: str, 
    context_refs: List[str]
) -> Dict[str, str]:
    '''キーで指定した項目の各期間の情報を一括取得'''
    results: Dict[str, str] = {}
    for context_ref in context_refs:
        extracted_data = xbrl.get_data_by_context_ref(key, context_ref)
        _date = translate_period(report_end_date, context_ref)
        results[_date] = extracted_data.get_value()
    return results

In [15]:
# 経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析 [テキストブロック]の取得
key = 'jpcrp_cor:ManagementAnalysisOfFinancialPositionOperatingResultsAndCashFlowsTextBlock'
context_ref = 'FilingDateInstant'
extracted_data = parsed_xbrl.get_data_by_context_ref(key, context_ref)
ManagementAnalysis = extracted_data.get_value()

AttributeError: 'NoneType' object has no attribute 'get_value'

In [16]:
extracted_data

In [ ]:
ManagementAnalysis

'<p style="page-break-before:always; line-height:0.75pt; width:100%; font-size:0.75pt;">\xa0</p>\n<h3 class="smt_head2" style="font-family:&apos;ＭＳ ゴシック&apos;;"><span style="font-family:&apos;ＭＳ ゴシック&apos;;">３ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】</span>\n</h3><p class="smt_text6" style="orphans:0;widows:0;text-align:justify;padding-left:9pt;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;text-indent:9pt;font-family:&apos;ＭＳ ゴシック&apos;;letter-spacing:0pt;line-height:12pt;font-size:9pt;"><span style="font-family:&apos;ＭＳ ゴシック&apos;;">当連結会計年度における当社グループ（当社、連結子会社及び持分法適用会社）の財政状態、経営成績及びキャッシュ・フロー（以下、「経営成績等」という。）の状況の概要並びに経営者の視点による当社グループの経営成績等の状況に関する認識及び分析・検討内容は次の通りであります。なお、文中の将来に関する事項は、当連結会計年度末現在において判断したものであります。</span></p><p class="smt_text6" style="orphans:0;widows:0;text-align:justify;padding-left:9pt;margin-bottom:0pt;text-indent:9pt;font-family:&apos;ＭＳ ゴシック&apos;;line-height:12pt;font-size:12pt;">\xa0</p>\n<h4 class="smt_head3" style="padding-left:9pt;margin-bottom:0pt;font-family:&apos;Ｍ

In [ ]:
# # 結果の出力
# from IPython.display import HTML
# HTML(ManagementAnalysis)

In [ ]:
def extract_paragraph_and_following_table(html_code, target_text):
    # BeautifulSoupを使ってパース
    soup = BeautifulSoup(html_code, 'html.parser')

    # 特定の文字列を含むパラグラフを抽出
    target_paragraphs = [p for p in soup.find_all('p') if target_text in p.text]

    # 対応する表を抽出
    tables = []
    for paragraph in target_paragraphs:
        table = paragraph.find_next('table')
        if table:
            tables.append(table)

    return target_paragraphs, tables

# 使用例
html_code = ManagementAnalysis

target_text = "主な相手先別"
paragraphs, tables = extract_paragraph_and_following_table(html_code, target_text)

# 結果を表示
for paragraph in paragraphs:
    print(paragraph.text)

for table in tables:
    print(table)


４  主な相手先別の販売実績及び当該販売実績の総販売実績に対する割合は次の通りであります。
<table cellpadding="0" cellspacing="0" class="align_left" style="border-collapse:collapse;border:solid 0pt #000000;width:432.6pt;table-layout:fixed;">
<colgroup>
<col style="width:141.4pt;min-width:141.4pt;"/>
<col style="width:72.8pt;min-width:72.8pt;"/>
<col style="width:72.8pt;min-width:72.8pt;"/>
<col style="width:72.8pt;min-width:72.8pt;"/>
<col style="width:72.8pt;min-width:72.8pt;"/>
</colgroup>
<tr style="height:17pt;min-height:17pt;">
<td rowspan="2" style="vertical-align:middle;border-top-style:solid;border-top-width:0.75pt;border-bottom-style:solid;border-bottom-width:0.75pt;border-left-style:solid;border-left-width:0.75pt;border-right-style:solid;border-right-width:0.75pt;overflow:hidden;" valign="middle"><p class="smt_tblC" style="orphans:0;widows:0;font-family:'ＭＳ ゴシック';"><span style="font-family:'ＭＳ ゴシック';">相手先</span></p>
</td>
<td colspan="2" style="vertical-align:middle;border-top-style:solid;border-top-width:0.75pt;border-

In [ ]:
for p in paragraphs:
    if '主な相手先別' in p.get_text():
        print(p.get_text())

４  主な相手先別の販売実績及び当該販売実績の総販売実績に対する割合は次の通りであります。


In [ ]:
def extract_table_content(table):
    # テーブルの行を取得
    rows = table.find_all('tr')

    # 各行のデータを取得
    table_data = []
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [col.text.strip() for col in cols]
        table_data.append(cols)

    return table_data

# 使用例
html_code = ManagementAnalysis

# 例として特定の文字列を含むパラグラフとその次に続く表を抽出する関数を呼び出す
target_text = "主な相手先別"
paragraphs, tables = extract_paragraph_and_following_table(html_code, target_text)

# 抽出したテーブルの内容を表示
for table in tables:
    table_content = extract_table_content(table)
    for row in table_content:
        print(row)


['相手先', '前連結会計年度', '当連結会計年度']
['金額(百万円)', '割合(％)', '金額(百万円)', '割合(％)']
['株式会社日本アクセス', '34,222', '18.7', '34,085', '18.0']


In [ ]:
label

,docID,edinetCode,secCode,filerName,docDescription
210,S100NN81,E00444,28110,カゴメ株式会社,有価証券報告書－第78期(令和3年1月1日－令和3年12月31日)


In [ ]:
# テーブルの内容をデータフレームに変換
additional_columns = ['相手先', '前連結_金額（百万円）', '前連結_割合（%）', '当連結_金額（百万円）', '当連結_割合（%）']

# データフレームを結合
label[additional_columns] = table_content[2]

C:\Users\hisan\AppData\Local\Temp\ipykernel_24060\200923689.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label[additional_columns] = table_content[2]
C:\Users\hisan\AppData\Local\Temp\ipykernel_24060\200923689.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label[additional_columns] = table_content[2]
C:\Users\hisan\AppData\Local\Temp\ipykernel_24060\200923689.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [ ]:
label

,docID,edinetCode,secCode,filerName,docDescription,相手先,前連結_金額（百万円）,前連結_割合（%）,当連結_金額（百万円）,当連結_割合（%）
210,S100NN81,E00444,28110,カゴメ株式会社,有価証券報告書－第78期(令和3年1月1日－令和3年12月31日),株式会社日本アクセス,"34,222",18.7,"34,085",18.0


In [ ]:
len(table_content[2])

5